In [1]:
import requests
from bs4 import BeautifulSoup
import numpy as np
from lxml import etree
import json, io
import re
from pathlib import Path


In [15]:
def jsonForcast(code):
    try:
        to_unicode = unicode
    except NameError:
        to_unicode = str
    urlBase = 'http://emweb.securities.eastmoney.com/PC_HSF10/ProfitForecast/ProfitForecastAjax?code='
    shOrSz = lambda x: ('sz' + code) if code[0] in {'0', '2', '3'} else 'sh' + code
    code = shOrSz(code)
    saveJsonStr = './data/forcast/%s.json' % code
    saveJson = Path(saveJsonStr)
    if (saveJson.exists() ):  # todo 更新
        with open(saveJsonStr, "r", encoding = "utf-8") as data_file:
            stockJson = json.load(data_file)
    else:
        html = requests.get(urlBase+code)
        stockJson = json.loads(html.text)
        saveJson.touch()
        with io.open(saveJsonStr, 'w', encoding='utf8') as outfile:
            str_ = json.dumps(stockJson,
                      indent=4, sort_keys=True,
                      separators=(',', ': '), ensure_ascii=False)
            outfile.write(to_unicode(str_))
    return stockJson

def zh2float(string):
    if('亿' == string[-1]):
        return float(string[:-1]) * 100000000
    elif('万' == string[-1]):
        return float(string[:-1]) * 10000

def jlrForcast(code):
    stockJson = jsonForcast(code)
    for i in stockJson['Result']['yctj']['data']:
        if (i['rq'].endswith('年预测')):
            if (i['jlr'] == '--'):
                return np.NaN
            jlr = zh2float(i['jlr'].split('(')[0])
#             print('预测： ' + str(jlr) + ' L: ' + str(jlrLast))
            nprg_forcast = round((jlr - jlrLast) / jlrLast * 100, 2)
            break
        jlrLast = zh2float(i['jlr'].split('(')[0])
    return nprg_forcast
    
    

In [27]:
from importlib import reload
import MyStock as ms

reload(ms)

test = ms.MyStock()

In [28]:
test.nprgForcast('300593')

2.04

In [29]:
test.nprgForcast('002614')

33.86

In [13]:
URL = ("http://data.eastmoney.com/bbsj/stock600057/yjbb.html")
html = requests.get(URL)
t = Path('test.html')
with t.open('w') as out:
    out.write(html.text)

In [6]:
from importlib import reload
import MyStock as ms

reload(ms)

<module 'MyStock' from 'D:\\Anaconda_code\\MyStock.py'>

In [32]:
test = ms.MyStock()

test.watch()

C:\Users\jlgs-jz\Anaconda3\envs\rqalpha\lib\site-packages\pandas\core\indexing.py:337: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\jlgs-jz\Anaconda3\envs\rqalpha\lib\site-packages\pandas\core\indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [33]:
test.stockGDF.head()

,SECUCODE,SECUNAME0,EPSJB0,EPSKCJB0,YS0,YSTZ0,YSHZ0,SJL0,SJLTZ0,SJLHZ0,...,rec_report_date,SJLTZ,pe_report_date,eps_ondate,peg_report_date,price_report_date,price_now,涨幅,pe,peg
30,300630,普利制药,0.27,0.26,1.324567e+08,19.13657395,-,2.479022e+07,61.755293,-,...,2017-03-06,37.625047,0.00,0.76,0.00,16.550,62.25,2.761329,99.17,2.64
125,603579,荣泰健康,0.52,0.36,4.114014e+08,25.98922032,-,2.582504e+07,281.829009,-,...,2017-02-27,44.808778,41.56,3.94,0.93,59.265,65.29,0.101662,44.33,0.99
192,603658,安图生物,0.33,0.3,4.093646e+08,28.6718068,-,1.240752e+08,29.080279,-,...,2017-03-03,25.800129,55.67,0.89,2.16,44.319,47.08,0.062298,45.82,1.78
334,300693,盛弘股份,-,-,1.271617e+08,82.95493791,-,2.202487e+07,51.827748,-,...,2017-07-31,48.497005,0.00,0.93,0.00,20.760,27.58,0.328516,50.04,1.03
377,300642,透景生命,-,-,7.682838e+07,57.72434061,-,2.257648e+07,65.782779,-,...,2017-03-14,48.628228,NaN,2.17,NaN,51.814,81.99,0.582391,49.38,1.02


In [34]:
test.stockGDF['nprg_forcast'] = test.stockGDF.apply(lambda x: test.nprgForcast(x['SECUCODE']), axis=1)

In [38]:
test.stockGDF['peg_forcast'] = test.stockGDF.apply(lambda x: np.NaN if np.isnan(x['nprg_forcast']) else x['pe']/x['nprg_forcast'], axis=1)

In [39]:
test.stockGDF[['SECUCODE','SECUNAME3', 'SJLTZ', 'peg_report_date', 'peg', 'pe_report_date', 'pe', 'price_now', 'rec_report_date', 'price_report_date', '涨幅']         
              + ['SJLTZ'+str(i) for i in range(5)]][(test.stockGDF.peg_forcast < 0.4) \
                                                                                    & (test.stockGDF.peg_forcast > 0)]

,SECUCODE,SECUNAME3,SJLTZ,peg_report_date,peg,pe_report_date,pe,price_now,rec_report_date,price_report_date,涨幅,SJLTZ0,SJLTZ1,SJLTZ2,SJLTZ3,SJLTZ4
1098,002078,太阳纸业,58.516953,0.28,0.28,16.51,16.21,12.29,2017-04-25,6.962,0.765297,61.402234,64.713334,42.200230,58.516953,NaN
1099,300061,康旗股份,111.907195,0.81,0.30,90.29,33.31,15.15,2017-04-25,18.660,-0.188103,2592.189196,75.192056,39.445351,111.907195,NaN
1213,002589,瑞康医药,150.386788,0.30,0.13,45.40,20.17,12.54,2017-03-31,17.790,-0.295110,29.736240,26.074940,30.362300,150.386788,NaN
1347,600477,杭萧钢构,272.381450,0.12,0.06,31.78,17.04,9.07,2017-04-15,10.325,-0.121550,138.597975,32.945987,103.365206,272.381450,NaN
1575,600326,西藏天路,73.029334,0.46,0.25,33.67,17.92,9.12,2017-03-28,9.682,-0.058046,111.521974,1207.604635,129.600610,73.029334,NaN
1580,300274,阳光电源,30.138789,0.90,0.76,27.25,23.03,15.00,2017-04-18,9.904,0.514540,148.664507,56.484144,50.166789,30.138789,NaN
1842,300116,坚瑞沃能,1102.977770,0.02,0.02,24.18,18.50,7.62,2017-09-27,11.440,-0.333916,137.224830,77.143374,313.535123,1102.977770,NaN
1945,300091,金通灵,39.464733,3.99,1.46,157.63,57.75,14.50,2017-04-21,14.379,0.008415,114.772033,448.862771,138.112377,39.464733,NaN
1973,300232,洲明科技,46.566671,0.94,0.65,43.95,30.45,13.35,2017-04-25,13.622,-0.019968,45.260076,84.579601,86.608863,46.566671,NaN
2465,002562,兄弟科技,103.865795,0.44,0.23,46.08,24.35,14.32,2017-03-08,14.526,-0.014181,132.489602,117.234321,105.438278,103.865795,NaN
